# Operating systems - Gantt charts and scheduling metrics

In this notebook, the idea of a Gantt chart will be intoduced and you will be introduced and asked to explore a set of scheduling metrics.

## Needed terms

We'll assume that, for a given set of processes, we are given the following information: 

* The name of the process, $P$
* The arrival time of the the process into the system, $T_{arrival}$
* The total service time of the process (how long it takes to complete the relevant work, assuming no breaks), $T_{service}$
* The time of first running of the process, $T_{firstrun}$
* The end time of the process, $T_{completion}$

As sanity checks, it should always be the case that for any given process $P$,

* $T_{firstrun} >= T_{arrival}$
* $T_{completion} >= T_{firstrun} + T_{service}$



## Gantt chart

In this course, we'll show execution of processes in the form of a Gantt chart.  In the Gantt chart, we'll list processes on the rows (on the vertical axis) and timesteps on the columns (the horizontal axis).  

Whenever a given process is executing in a timestep, an X will be placed in the corresponding (row, column) entry; otherwise, the entry is left blank (represented by a dash in the table below).

An example Gantt chart with three processes over 12 timesteps is below:

| Process | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 |
|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|
| A | X | X | X | X | X | X | X | - | - | - | - | - | 
| B | - | - | - | - | - | - | - | - | X | X | X | X | 
| C | - | - | - | - | - | - | - | X | - | - | - | - |


Gantt charts are commonly used in project planning, where the tasks to be completed are represented on the vertical axis and the planned timelines for each activity are represented on the horizontal axis.  In project management scenarios, it is possible to have multiple tasks in execution at the same time - an idea we won't support for the moment, given an assumption that we are working with only one CPU and can only execute one process at a time.  Since we are assuming no two processes are executing on the CPU at the same time, we could also collapse our representation into a linear representation:

| Time | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 |
|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|
| Process | A | A | A | A | A | A | A | C | B | B | B | B |

Since these charts show execution times, we can extract $T_{firstrun}$, $T_{completion}$, and $T_{service}$, but we can't determine when each process arrived $T_{firstarrival}$.

## Metric definitions

### User-focused metrics

The first metrics we'll look at should be considered *user-focused*, as they relate to a user's perception of how his or her processes are being executed. The metric of *response time* is a particularly good example of such a metric. If one thinks of a process as generating output during execution, response time provides insight into how long it takes between the process entering the system and the first output to the user (assuming output from the process starts when the process starts executing). Low response time is important for interactive systems.

The metrics of interest are: 

* *Turn-around time*: For a given process, the amount of time elapsed between process arrival and process completion.  $TA = T_{completion} - T_{arrival}$
* *Normalized turn-around time*: For a given process, the ratio of turn-around time to service time.  $NTA = TA / T_{service}$
* *Response time*: For a given process, the amount of time elapsed between process arrival and the time when the process first runs.  $R = T_{firstrun} - T_{arrival}$
* *Average turn-around time*: For a set of N processes, the average turn-around time for the processes in that set.
* *Average response time*: For a set of N processes, the average response time for the processes in that set.

Note that these metrics are distinct from metrics based on the *state trace* of a process (where we had metrics such as % of time in the *Blocked* state).


### Examples
Consider the set of processes described in the table below.

|Process|$T_{arrival}$|$T_{service}$|$T_{firstrun}$|$T_{completion}$|
|:--:|:--:|:--:|:--:|:--:|
|0|0|3|0|3|
|1|2|6|3|9|
|2|4|4|9|13|
|3|6|5|13|18|
|4|8|2|18|20|


You can assume that processes complete and begin at the turn of the clock.  As an example, Process 0 begins execution at time 0; executes through timestep 0, 1, and 2; and ends execution as timestep 2 turns into timestep 3, at which point Process 1 begins.

For this set of processes, the metrics of interest are in the table below.  We'll assume the unit of measurement for TA and R is arbitrary "clock units".

|Job|TA|NTA|R|
|:--:|:--:|:--:|:--:|
|0|$3-0 = 3$|$3/3 = 1.00$|$0-0 = 0$|
|1|$9-2 = 7$|$7/6 = 1.17$|$3-2 = 1$|
|2|$13-4 = 9$|$9/4 = 2.25$|$9-4 = 5$|
|3|$18-6 = 12$|$12/5 = 2.40$|$13-6 = 7$|
|4|$20-8 = 12$|$12/2 = 6.00$|$18-8 = 10$|

The average turn-around time is: $(3+7+9+12+12) / 5 = 43 / 5 = 8.60$ clock units.

The average response time is: $(0+1+5+7+10) / 5 = 23 / 5 = 4.60$ clock units.


### System-focused metrics
Other metrics are considered *system-focused*, as they relate to how the system perceives is making use of its resources. Initially, we will look at just one of these metrics.

* Throughput: The number of processes completed  per unit of time.  For a set of N processes, $N / (max(T_{completion}) - min(T_{arrival}))$

The throughput for the five processes in the example above is: $5 / (20-0) = 0.25$ processes per clock unit.

## Metric computations code

The Python code in the cell below will compute the metrics outlined above for you. Feel free to experiment with the process definitions and re-run the cell.

In [ ]:
# The variable processes represents a dictionary (hashtable), 
# where the key is a process id (0-4)
# and the values are also dictionaries, with the keys 
# being strings representing a process
# feature and the values being an actual time associated 
# with that feature.  The data is currently representing the 
# processes in the table outlined above under the *Examples* section

processes = {}
processes[0] = {'arrival':0,'service':3,'firstrun':0,'completion':3}
processes[1] = {'arrival':2,'service':6,'firstrun':3,'completion':9}
processes[2] = {'arrival':4,'service':4,'firstrun':9,'completion':13}
processes[3] = {'arrival':6,'service':5,'firstrun':13,'completion':18}
processes[4] = {'arrival':8,'service':2,'firstrun':18,'completion':20}

# These variables help compute the summary metrics; they are
# initialized to 0 and then added to as each process executes
turnaroundSum = 0
responseSum = 0
# This determines the total number of jobs that are executed by
# determining the length of the jobs dictionary
numberOfProcesses = len(processes)
# These variables are used to hold the largest time of completion 
# and smallest time of arrival across the set of processes
maxTimeCompletion = -1
minTimeArrival = -1

# For each process
for processName in processes:
    # Get the data associatd with the job name 
    # and store in the processInfo variable
    processInfo = processes[processName]
    # If the completion time for this process is larger 
    # than the largest seen so far,
    # it is the new maximum completion time; similarly, 
    # check for min arrival time
    if (processName == 0):
        maxTimeCompletion = processInfo['completion']
        minTimeArrival = processInfo['arrival']
    else:
        if (processInfo['completion'] > maxTimeCompletion):
            maxTimeCompletion = processInfo['completion']
        if (processInfo['arrival'] < minTimeArrival):
            minTimeArrival = processInfo['arrival']
    # Compute turnaround time for this process
    turnaroundTime = processInfo['completion'] - processInfo['arrival']
    # Compute normalized turnaround time for this process
    normalizedTurnaroundTime = turnaroundTime / processInfo['service']
    # Compute response time for this process
    responseTime = processInfo['firstrun']-processInfo['arrival']
    # Print the three metrics for this process
    print("Process ",processName,"- TA: ",turnaroundTime, "NTA: ",normalizedTurnaroundTime," R: ",responseTime)
    # Add this process' turnaround time to the sum over all processes
    turnaroundSum += turnaroundTime
    # Add this process' response time to the sum over all processes
    responseSum += responseTime

# Compute and print the metrics that take into account all processes
print("Average turn-around time: ", (turnaroundSum/numberOfProcesses))
print("Average response time: ", (responseSum/numberOfProcesses))
print("Throughput: ", (numberOfProcesses/(maxTimeCompletion-minTimeArrival)))